# Final Data Frame Merging

In [16]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import json

from functools import reduce

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

import warnings

We load all data frames as "pickles" generated from the previous notebooks. To simplify the further process, we reference the data frames to a list.

In [17]:
df_merged_1 = pd.read_pickle('../data/pickle/df_merged_1.pickle')
df_merged_2 = pd.read_pickle('../data/pickle/df_merged_2.pickle')
df_merged_3 = pd.read_pickle('../data/pickle/df_merged_3.pickle')


dfs = [df_merged_1, df_merged_2, df_merged_3]

All data frames will be merged to a single data frame

In [18]:
df_merged_4 = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],
                                            how='inner'), dfs)

/tmp/ipykernel_353/3814285556.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'DK1_power_mw_y', 'DK_power_mw_y', 'TTG_power_mw_y', 'DE_power_mw_y', '50Hertz_power_mw_y'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged_4 = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],


In [19]:
count_inf = np.isinf(df_merged_4).values.sum()
count_nan = df_merged_4.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 192 missing values.


In [20]:
df_merged_4.shape

(17368, 64)

In [21]:
df_merged_4.head()

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Biomasse[MWh],Wasserkraft[MWh],Wind Offshore[MWh],Wind Onshore[MWh],...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,1392.0,3378.0,184.0,70.0,1248.0,6272.0,NaN,NaN,NaN,NaN,...,250.0,483.0,268.0,136.0,233.0,1616.0,3821.0,227.0,84.0,1437.0
2021-01-01 00:15:00,1342.0,3267.0,184.0,70.0,1198.0,6061.0,NaN,NaN,NaN,NaN,...,238.0,469.0,268.0,136.0,231.0,1552.0,3708.0,227.0,84.0,1391.0
2021-01-01 00:30:00,1299.0,3164.0,184.0,70.0,1148.0,5865.0,NaN,NaN,NaN,NaN,...,226.0,458.0,268.0,136.0,232.0,1495.0,3598.0,227.0,84.0,1344.0
2021-01-01 00:45:00,1258.0,3063.0,184.0,70.0,1099.0,5674.0,NaN,NaN,NaN,NaN,...,214.0,449.0,268.0,136.0,235.0,1442.0,3488.0,227.0,84.0,1297.0
2021-01-01 01:00:00,1219.0,2969.0,153.0,61.0,1054.0,5456.0,1.147,308.0,84.0,1.058,...,202.0,438.0,214.0,101.0,236.0,1392.0,3378.0,184.0,70.0,1248.0


The "NaNs" values are created by shifting rows forward. We'll drop these rows from our data frame

In [22]:
df_merged_4.dropna(inplace=True)

In [23]:
df_merged_4.shape

(17358, 64)

In [24]:
count_inf = np.isinf(df_merged_4).values.sum()
count_nan = df_merged_4.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 0 missing values.


In [26]:
df_merged_4.to_pickle('../data/pickle/df_merged_4.pickle')